# Imports

In [9]:
# Raw Package
import numpy as np
import pandas as pd

#Data Source
import yfinance as yf

import talib
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

import datetime
# import time

# ignore warning
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Functions

In [14]:
def read_ticker_list(ticker_list):
    ticker_list = pd.read_csv("Ticker List/" + ticker_list).dropna()
    ticker_list = ticker_list['Symbol'].tolist()
    return(ticker_list)

def calculate_financial_ratios(temp_ticker, term = 365, period = 'annual'):
    try:
        if(period == 'annual'):
            test_income_statement = yf.Ticker(temp_ticker).financials/10000000
            test_balance_sheet = yf.Ticker(temp_ticker).balance_sheet/10000000
        elif(period == 'quarterly'):
            test_income_statement = yf.Ticker(temp_ticker).quarterly_financials/10000000
            test_balance_sheet = yf.Ticker(temp_ticker).quarterly_balance_sheet/10000000
        temp_gen_info = yf.Ticker(temp_ticker).info
        
        # Current and Previous year financials
        bs_curr_year = test_balance_sheet.iloc[:,0]
        is_curr_year = test_income_statement.iloc[:,0]
        # bs_prev_year = test_balance_sheet.iloc[:,1]
        # is_prev_year = test_income_statement.iloc[:,1]
    except Exception as e:
        print(temp_ticker, ' Could not get data')
        print(e)

    fr_dict = {}

    try:
        # Balance Sheet
        try:
            current_assets = bs_curr_year['Current Assets']
        except:
            current_assets = np.nan
        #current_assets = bs_curr_year['Total Current Assets']

        try:
            current_liabilities = bs_curr_year['Current Liabilities']
        except:
            current_liabilities = np.nan  
        #current_liabilities = bs_curr_year['Total Current Liabilities']

        try:
            cash = bs_curr_year['Cash And Cash Equivalents']
        except:
            cash = np.nan 
        #cash = bs_curr_year['Cash']

        try:
            accounts_receivable = bs_curr_year['Receivables']
        except:
            accounts_receivable = np.nan

        try:
            inventory = bs_curr_year['Accounts Payable']
        except:
            inventory = np.nan
        
        try:
            accounts_payable = bs_curr_year['Accounts Payable']
        except:
            accounts_payable = np.nan

        total_assets = bs_curr_year['Total Assets']
        total_liabilities = bs_curr_year['Total Liabilities Net Minority Interest']
        ppe = bs_curr_year['Net PPE']

        # Income Statement
        revenue = is_curr_year['Total Revenue']    
        try:
            cogs = is_curr_year['Cost Of Revenue']
        except:
            cogs = np.nan
        try:
            gross_profit = is_curr_year['Gross Profit']
        except:
            gross_profit = np.nan
        try:
            operating_income = is_curr_year['Operating Income']
        except:
            operating_income = np.nan
        income_before_tax = is_curr_year['Pretax Income']
        net_income = is_curr_year['Net Income']
        try:
            income_tax_expense = is_curr_year['Operating Expense']
        except:
            income_tax_expense = np.nan

        try:
            ebit = is_curr_year['EBIT']
        except:
            ebit = np.nan
        try:
            interest_expense = abs(is_curr_year['Interest Expense'])
        except:
            interest_expense = np.nan

        # Liquidity Measurement Ratios
        fr_dict['current_ratio'] = current_assets/current_liabilities # A current ratio of 1.0 or greater is an indication that the company is well-positioned to cover its current or short-term liabilities.
        fr_dict['DSO'] = (accounts_receivable/revenue)*term #DSO tells you how many days after the sale it takes people to pay you on average.
        try:
            fr_dict['DIO'] = (inventory/cogs)*term #DIO tells you how many days inventory sits on the shelf on average.
        except:
            fr_dict['DIO'] = np.nan
        fr_dict['operating_cycle'] = fr_dict['DSO'] + fr_dict['DIO'] # (DSO + DIO )Basically the Operating Cycle tells you how many days it takes for something to go from first being in inventory to receiving the cash after the sale.
        try:
            fr_dict['DPO'] = (accounts_payable/cogs)*term #DPO tells you how many days the company takes to pay its suppliers.
        except:
            fr_dict['DPO'] = np.nan
        fr_dict['CCC'] = fr_dict['operating_cycle'] - fr_dict['DPO'] #The cash conversion cycle (CCC = DSO + DIO – DPO) measures the number of days a company's cash is tied up in the production and sales process of its operations and the benefit it derives from payment terms from its creditors. The shorter this cycle, the more liquid the company's working capital position is. The CCC is also known as the "cash" or "operating" cycle.

        # Profitability Indicator Ratios
        fr_dict['gross_profit_margin'] = gross_profit / revenue # You can think of it as the amount of money from product sales left over after all of the direct costs associated with manufacturing the product have been paid.
        fr_dict['operating_profit_margin'] = operating_income / revenue # If companies can make enough money from their operations to support the business, the company is usually considered more stable.
        fr_dict['pretax_profit_margin'] = income_before_tax / revenue #Profit is the main goal of for-profit organizations. The goal is to make a profit through growth and to grow every year. As a result, one of the most important roles of the financial and investment analyst is to track and forecast profitability.
        fr_dict['net_profit_margin'] = net_income / revenue # Generally, a net profit margin in excess of 10% is considered excellent, though it depends on the industry and the structure of the business.
        fr_dict['effective_tax_rate'] = income_tax_expense / income_before_tax # If there’s one takeaway, it should be that a company’s tax situation is all but a living, breathing organism in its own right.
        fr_dict['return_on_assets'] = net_income / total_assets # ROA Return on assets gives an indication of the capital intensity of the company, which will depend on the industry; companies that require large initial investments will generally have lower return on assets. ROAs over 5% are generally considered good.
        fr_dict['ROCE'] = ebit / (total_assets - current_liabilities) # ROCE shows investors how many dollars in profits each dollar of capital employed generates.

        # Debt Ratios
        fr_dict['debt_ratio'] = total_liabilities / total_assets #T he debt ratio tells us the degree of leverage used by the company.
        fr_dict['interest_coverage_ratio'] = ebit / interest_expense # The lower a company’s interest coverage ratio is, the more its debt expenses burden the company.

        # Operating Performance Ratios
        fr_dict['fixed_asset_turnover'] = revenue / ppe # Calculates how efficiently a company is a producing sales with its machines and equipment.
        fr_dict['asset_turnover'] = revenue / total_assets # The Asset Turnover ratio can often be used as an indicator of the efficiency with which a company is deploying its assets in generating revenue.


        #in-built ratios
        try:
            fr_dict['twoHundredDayAverage'] = temp_gen_info['twoHundredDayAverage']
        except:
            fr_dict['twoHundredDayAverage'] = np.nan
        #fr_dict['twoHundredDayAverage'] = temp_gen_info['twoHundredDayAverage']

        try:
            fr_dict['payoutRatio'] = temp_gen_info['payoutRatio']
        except:
            fr_dict['payoutRatio'] = np.nan
        #fr_dict['payoutRatio'] = temp_gen_info['payoutRatio']

        try:
            fr_dict['fiftyDayAverage'] = temp_gen_info['fiftyDayAverage']
        except:
            fr_dict['fiftyDayAverage'] = np.nan
        #fr_dict['fiftyDayAverage'] = temp_gen_info['fiftyDayAverage']

        try:
            fr_dict['trailingAnnualDividendRate'] = temp_gen_info['trailingAnnualDividendRate']
        except:
            fr_dict['trailingAnnualDividendRate'] = np.nan
        #fr_dict['trailingAnnualDividendRate'] = temp_gen_info['trailingAnnualDividendRate']

        try:
            fr_dict['dividendRate'] = temp_gen_info['dividendRate']
        except:
            fr_dict['dividendRate'] = np.nan
        #fr_dict['dividendRate'] = temp_gen_info['dividendRate']

        try:
            fr_dict['trailing_PE'] = temp_gen_info['trailingPE']
        except:
            fr_dict['trailing_PE'] = np.nan
        #fr_dict['trailing_PE'] = temp_gen_info['trailingPE']

        try:
            fr_dict['market_cap'] = temp_gen_info['marketCap']
        except:
            fr_dict['market_cap'] = np.nan
        #fr_dict['market_cap'] = temp_gen_info['marketCap']

        try:
            fr_dict['priceToSalesTrailing12Months'] = temp_gen_info['priceToSalesTrailing12Months']
        except:
            fr_dict['priceToSalesTrailing12Months'] = np.nan
        #fr_dict['priceToSalesTrailing12Months'] = temp_gen_info['priceToSalesTrailing12Months']

        try:
            fr_dict['forward_PE'] = temp_gen_info['forwardPE']
        except:
            fr_dict['forward_PE'] = np.nan
        #fr_dict['forward_PE'] = temp_gen_info['forwardPE']

        try:
            fr_dict['fiftyTwoWeekHigh'] = temp_gen_info['fiftyTwoWeekHigh']
        except:
            fr_dict['fiftyTwoWeekHigh'] = np.nan
        #fr_dict['fiftyTwoWeekHigh'] = temp_gen_info['fiftyTwoWeekHigh']

        try:
            fr_dict['fiftyTwoWeekLow'] = temp_gen_info['fiftyTwoWeekLow']
        except:
            fr_dict['fiftyTwoWeekLow'] = np.nan
        #fr_dict['fiftyTwoWeekLow'] = temp_gen_info['fiftyTwoWeekLow']

        try:
            fr_dict['enterpriseToRevenue'] = temp_gen_info['enterpriseToRevenue']
        except:
            fr_dict['enterpriseToRevenue'] = np.nan
        #fr_dict['enterpriseToRevenue'] = temp_gen_info['enterpriseToRevenue']

        try:
            fr_dict['profitMargins'] = temp_gen_info['profitMargins']
        except:
            fr_dict['profitMargins'] = np.nan
        #fr_dict['profitMargins'] = temp_gen_info['profitMargins']

        try:
            fr_dict['enterpriseToEbitda'] = temp_gen_info['enterpriseToEbitda']
        except:
            fr_dict['enterpriseToEbitda'] = np.nan
        #fr_dict['enterpriseToEbitda'] = temp_gen_info['enterpriseToEbitda']

        try:
            fr_dict['trailing_EPS'] = temp_gen_info['trailingEps']
        except:
            fr_dict['trailing_EPS'] = np.nan
        try:
            fr_dict['forward_EPS'] = temp_gen_info['forwardEps']
        except:
            fr_dict['forward_EPS'] = np.nan
        try:
            fr_dict['bookValue'] = temp_gen_info['bookValue']
        except:
            fr_dict['bookValue'] = np.nan
        try:
            fr_dict['priceToBook'] = temp_gen_info['priceToBook']
        except:
            fr_dict['priceToBook'] = np.nan
        try:
            fr_dict['cmp'] = temp_gen_info['regularMarketPreviousClose']
        except:
            fr_dict['cmp'] = np.nan
    except Exception as e:
        print(temp_ticker," Errored out")
        print(e)
    return(fr_dict)

def create_df_of_financial_ratios(csv_file = 'n200.csv', period = 'annual'):
    frames = []
    n50 = pd.read_csv("Ticker List/" + csv_file)
    n50 = n50[['Industry', 'Symbol']]
    n50['Symbol'] = n50['Symbol'].apply(lambda x: x + '.NS')
    i=1
    for sector, t in n50.values:
        print(i, t)
        i = i + 1
        ratio_data_single_ticker = calculate_financial_ratios(t, period = period)
        temp_df = pd.DataFrame(ratio_data_single_ticker, index=[t])
        temp_df['Sector'] = sector
        frames.append(temp_df)
    return(pd.concat(frames))

# create lists of ticker symbols
n50 = read_ticker_list('n50.csv')
n200 = read_ticker_list('n200.csv')
n500 = read_ticker_list('n500.csv')

# Main

In [15]:
start = datetime.datetime.now()
combined_financial_ratio_df = create_df_of_financial_ratios('n500.csv', period = 'quarterly')
print('Time taken : ',datetime.datetime.now() - start)

combined_financial_ratio_df

1 3MINDIA.NS
2 ABB.NS
3 POWERINDIA.NS
4 ACC.NS
5 AIAENG.NS
6 APLAPOLLO.NS
7 AUBANK.NS
8 AARTIDRUGS.NS
9 AARTIIND.NS
10 AAVAS.NS
11 ABBOTINDIA.NS
12 ADANIENT.NS
13 ADANIGREEN.NS
14 ADANIPORTS.NS
15 ATGL.NS
16 ADANITRANS.NS
17 ABCAPITAL.NS
18 ABFRL.NS
19 ADVENZYMES.NS
20 AEGISCHEM.NS
21 AFFLE.NS
22 AJANTPHARM.NS
23 AKZOINDIA.NS
24 ALEMBICLTD.NS
25 APLLTD.NS
26 ALKEM.NS
27 ALKYLAMINE.NS
28 ALOKINDS.NS
29 AMARAJABAT.NS
30 AMBER.NS
31 AMBUJACEM.NS
32 APOLLOHOSP.NS
33 APOLLOTYRE.NS
34 ASAHIINDIA.NS
35 ASHOKLEY.NS
36 ASHOKA.NS
37 ASIANPAINT.NS
38 ASTERDM.NS
39 ASTRAZEN.NS
40 ASTRAL.NS
41 ATUL.NS
42 AUROPHARMA.NS
43 AVANTIFEED.NS
44 DMART.NS
45 AXISBANK.NS
46 BASF.NS
47 BEML.NS
48 BSE.NS
49 BAJAJ-AUTO.NS
50 BAJAJCON.NS
51 BAJAJELEC.NS
52 BAJFINANCE.NS
53 BAJAJFINSV.NS
54 BAJAJHLDNG.NS
55 BALAMINES.NS
56 BALKRISIND.NS
57 BALMLAWRIE.NS
58 BALRAMCHIN.NS
59 BANDHANBNK.NS
60 BANKBARODA.NS
61 BANKINDIA.NS
62 MAHABANK.NS
63 BATAINDIA.NS
64 BAYERCROP.NS
65 BERGEPAINT.NS
66 BDL.NS
67 BEL.NS
68 BHARATFO

,current_ratio,DSO,DIO,operating_cycle,DPO,CCC,gross_profit_margin,operating_profit_margin,pretax_profit_margin,net_profit_margin,...,fiftyTwoWeekLow,enterpriseToRevenue,profitMargins,enterpriseToEbitda,trailing_EPS,forward_EPS,bookValue,priceToBook,cmp,Sector
3MINDIA.NS,2.170101,NaN,438.967902,NaN,438.967902,NaN,0.392344,0.172266,0.176201,0.129746,...,21300.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28590.45,CONSUMER GOODS
ABB.NS,1.815628,NaN,626.741122,NaN,626.741122,NaN,0.359087,0.107611,0.136836,0.102210,...,2640.00,10.230,0.13122,84.382,41.98,5.03,233.092,18.910988,4519.65,INDUSTRIAL MANUFACTURING
POWERINDIA.NS,1.157280,NaN,871.047299,NaN,871.047299,NaN,0.367956,0.011363,0.003366,0.002400,...,2840.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3964.85,INDUSTRIAL MANUFACTURING
ACC.NS,1.524411,NaN,285.392271,NaN,285.392271,NaN,0.509397,0.109752,0.120272,0.089615,...,1592.35,NaN,0.00000,NaN,43.45,106.62,752.898,2.618216,2010.60,CEMENT & CEMENT PRODUCTS
AIAENG.NS,6.071437,NaN,165.127902,NaN,165.127902,NaN,0.538483,0.233659,0.276726,0.214243,...,2358.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3376.35,INDUSTRIAL MANUFACTURING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YESBANK.NS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.145006,0.108747,...,14.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.10,FINANCIAL SERVICES
ZEEL.NS,4.265314,NaN,485.819657,NaN,485.819657,NaN,0.374900,0.072801,0.017188,-0.093228,...,170.10,2.847,0.00591,20.381,2.58,12.27,111.626,2.031337,237.05,MEDIA ENTERTAINMENT & PUBLICATION
ZENSARTECH.NS,2.701468,NaN,110.673813,NaN,110.673813,NaN,0.255052,0.153031,0.171284,0.127282,...,201.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,491.00,IT
ZYDUSWELL.NS,1.311121,NaN,326.130789,NaN,326.130789,NaN,0.506071,0.194267,0.177955,0.204719,...,1370.10,4.232,0.13765,28.742,48.95,NaN,805.041,1.833074,1479.75,CONSUMER GOODS


In [16]:
imp_financial_df = combined_financial_ratio_df[['ROCE', 'debt_ratio', 'operating_profit_margin', 'net_profit_margin', 'trailing_PE', 'forward_PE',
                         'trailing_EPS', 'forward_EPS', 'enterpriseToEbitda', 'fixed_asset_turnover',
                         'enterpriseToRevenue', 'market_cap', 'payoutRatio', 'cmp', 'fiftyTwoWeekHigh', 
                         'fiftyTwoWeekLow', 'twoHundredDayAverage', 'fiftyDayAverage','Sector']].dropna(thresh=10)
imp_financial_df

,ROCE,debt_ratio,operating_profit_margin,net_profit_margin,trailing_PE,forward_PE,trailing_EPS,forward_EPS,enterpriseToEbitda,fixed_asset_turnover,enterpriseToRevenue,market_cap,payoutRatio,cmp,fiftyTwoWeekHigh,fiftyTwoWeekLow,twoHundredDayAverage,fiftyDayAverage,Sector
3MINDIA.NS,0.109623,0.382016,0.172266,0.129746,70.805950,NaN,NaN,NaN,NaN,2.887766,NaN,3.184323e+11,NaN,28590.45,29190.00,21300.45,23958.40400,27065.475,CONSUMER GOODS
ABB.NS,0.066443,0.469920,0.107611,0.102210,105.002380,876.341900,41.98,5.03,84.382,2.520848,10.230,9.506336e+11,0.1224,4519.65,4614.35,2640.00,3447.24100,4283.333,INDUSTRIAL MANUFACTURING
POWERINDIA.NS,0.011352,0.689854,0.011363,0.002400,180.196610,NaN,NaN,NaN,NaN,1.377236,NaN,1.717126e+11,NaN,3964.85,4365.00,2840.10,3452.04050,4066.035,INDUSTRIAL MANUFACTURING
ACC.NS,0.044616,0.326205,0.109752,0.089615,45.368240,18.488558,43.45,106.62,NaN,0.591810,NaN,3.701751e+11,1.3367,2010.60,2785.00,1592.35,2025.51720,1826.977,CEMENT & CEMENT PRODUCTS
AIAENG.NS,0.061899,0.140387,0.233659,0.214243,30.543507,30.470978,NaN,NaN,NaN,1.151527,NaN,3.267777e+11,NaN,3376.35,3617.60,2358.00,2855.07570,3266.197,INDUSTRIAL MANUFACTURING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YESBANK.NS,NaN,0.884536,NaN,0.108747,58.620693,18.279570,NaN,NaN,NaN,1.257525,NaN,4.891206e+11,NaN,17.10,24.75,14.40,17.04325,16.621,FINANCIAL SERVICES
ZEEL.NS,0.006801,0.218998,0.072801,-0.093228,87.887596,18.480032,2.58,12.27,20.381,2.877737,2.847,2.177977e+11,1.1450,237.05,287.00,170.10,218.09875,199.835,MEDIA ENTERTAINMENT & PUBLICATION
ZENSARTECH.NS,0.066262,0.277797,0.153031,0.127282,26.821726,NaN,NaN,NaN,NaN,3.762109,NaN,1.090648e+11,NaN,491.00,504.45,201.50,291.58676,409.131,IT
ZYDUSWELL.NS,0.025578,0.121789,0.194267,0.204719,30.147087,NaN,48.95,NaN,28.742,2.505647,4.232,9.403573e+10,0.1025,1479.75,1791.00,1370.10,1519.09350,1481.119,CONSUMER GOODS


In [17]:
# Rank for "higher the better metrics"
for col in ['ROCE', 'operating_profit_margin', 'net_profit_margin', 'trailing_EPS', 'fixed_asset_turnover', 'market_cap', 'payoutRatio']:
    imp_financial_df['score_' + col] = imp_financial_df[col].rank()

# Rank for "lower the better metrics  "  
for col in ['debt_ratio', 'trailing_PE', 'enterpriseToEbitda', 'enterpriseToRevenue']:
    imp_financial_df['score_' + col] = imp_financial_df[col].rank(ascending  = False)

# Custom ranks
imp_financial_df['score_close_to_52_week_high'] = (imp_financial_df['cmp'] / imp_financial_df['fiftyTwoWeekHigh']).rank(ascending  = False)
    

imp_financial_score_df = imp_financial_df.drop(columns = ['ROCE', 'debt_ratio', 'operating_profit_margin', 'net_profit_margin', 'trailing_PE', 'forward_PE',
                         'trailing_EPS', 'forward_EPS', 'enterpriseToEbitda', 'fixed_asset_turnover',
                         'enterpriseToRevenue', 'market_cap', 'payoutRatio', 'cmp', 'fiftyTwoWeekHigh', 
                         'fiftyTwoWeekLow', 'twoHundredDayAverage', 'fiftyDayAverage'])
imp_financial_score_df['final_score'] = imp_financial_score_df.drop(columns = ['Sector']).mean(axis = 1)
imp_financial_score_df.sort_values('final_score', ascending=False)

,Sector,score_ROCE,score_operating_profit_margin,score_net_profit_margin,score_trailing_EPS,score_fixed_asset_turnover,score_market_cap,score_payoutRatio,score_debt_ratio,score_trailing_PE,score_enterpriseToEbitda,score_enterpriseToRevenue,score_close_to_52_week_high,final_score
NMDC.NS,METALS,406.0,389.0,456.0,NaN,270.0,329.0,NaN,431.0,428.0,NaN,NaN,352.0,382.625000
OFSS.NS,IT,343.0,400.0,449.0,NaN,444.0,328.0,NaN,433.0,324.0,NaN,NaN,165.0,360.750000
FINEORG.NS,CHEMICALS,378.0,348.0,362.0,NaN,345.0,211.0,NaN,450.0,290.0,NaN,NaN,465.0,356.125000
NAM-INDIA.NS,FINANCIAL SERVICES,370.0,413.0,474.0,NaN,473.0,256.0,NaN,471.0,261.0,NaN,NaN,115.0,354.125000
TATAELXSI.NS,IT,396.0,359.0,395.0,NaN,370.0,358.0,NaN,412.0,85.0,NaN,NaN,455.0,353.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BIRLACORPN.NS,CEMENT & CEMENT PRODUCTS,72.0,83.0,93.0,NaN,39.0,156.0,NaN,183.0,5.0,NaN,NaN,58.0,86.125000
CENTURYTEX.NS,PAPER AND JUTE,29.0,91.0,38.0,60.0,57.0,185.0,10.0,214.0,117.0,45.0,74.0,108.0,85.666667
IDEA.NS,TELECOM,20.0,17.0,8.0,3.0,20.0,344.0,10.0,4.0,NaN,38.0,34.0,371.0,79.000000
DCAL.NS,PHARMA,22.0,31.0,21.0,NaN,25.0,13.0,NaN,299.0,NaN,NaN,NaN,44.0,65.000000


In [22]:
imp_financial_score_df.sort_values('final_score', ascending=False).head(10)

,Sector,score_ROCE,score_operating_profit_margin,score_net_profit_margin,score_trailing_EPS,score_fixed_asset_turnover,score_market_cap,score_payoutRatio,score_debt_ratio,score_trailing_PE,score_enterpriseToEbitda,score_enterpriseToRevenue,score_close_to_52_week_high,final_score
NMDC.NS,METALS,406.0,389.0,456.0,NaN,270.0,329.0,NaN,431.0,428.0,NaN,NaN,352.0,382.625000
OFSS.NS,IT,343.0,400.0,449.0,NaN,444.0,328.0,NaN,433.0,324.0,NaN,NaN,165.0,360.750000
FINEORG.NS,CHEMICALS,378.0,348.0,362.0,NaN,345.0,211.0,NaN,450.0,290.0,NaN,NaN,465.0,356.125000
NAM-INDIA.NS,FINANCIAL SERVICES,370.0,413.0,474.0,NaN,473.0,256.0,NaN,471.0,261.0,NaN,NaN,115.0,354.125000
TATAELXSI.NS,IT,396.0,359.0,395.0,NaN,370.0,358.0,NaN,412.0,85.0,NaN,NaN,455.0,353.750000
IEX.NS,FINANCIAL SERVICES,401.0,419.0,475.0,NaN,420.0,181.0,NaN,255.0,192.0,NaN,NaN,437.0,347.500000
EICHERMOT.NS,AUTOMOBILE,340.0,332.0,408.0,NaN,309.0,420.0,NaN,422.0,225.0,NaN,NaN,324.0,347.500000
HDFCAMC.NS,FINANCIAL SERVICES,372.0,418.0,478.0,NaN,422.0,379.0,NaN,476.0,190.0,NaN,NaN,36.0,346.375000
TATAINVEST.NS,FINANCIAL SERVICES,NaN,NaN,471.0,NaN,470.0,200.0,NaN,474.0,127.0,NaN,NaN,323.0,344.166667
IRCTC.NS,SERVICES,404.0,384.0,427.0,NaN,374.0,375.0,NaN,220.0,123.0,NaN,NaN,362.0,333.625000


In [21]:
imp_financial_score_df.sort_values('final_score', ascending=True).head(10)

,Sector,score_ROCE,score_operating_profit_margin,score_net_profit_margin,score_trailing_EPS,score_fixed_asset_turnover,score_market_cap,score_payoutRatio,score_debt_ratio,score_trailing_PE,score_enterpriseToEbitda,score_enterpriseToRevenue,score_close_to_52_week_high,final_score
GMRINFRA.NS,CONSTRUCTION,32.0,6.0,14.0,NaN,5.0,318.0,NaN,9.0,NaN,NaN,NaN,46.0,61.428571
DCAL.NS,PHARMA,22.0,31.0,21.0,NaN,25.0,13.0,NaN,299.0,NaN,NaN,NaN,44.0,65.000000
IDEA.NS,TELECOM,20.0,17.0,8.0,3.0,20.0,344.0,10.0,4.0,NaN,38.0,34.0,371.0,79.000000
CENTURYTEX.NS,PAPER AND JUTE,29.0,91.0,38.0,60.0,57.0,185.0,10.0,214.0,117.0,45.0,74.0,108.0,85.666667
BIRLACORPN.NS,CEMENT & CEMENT PRODUCTS,72.0,83.0,93.0,NaN,39.0,156.0,NaN,183.0,5.0,NaN,NaN,58.0,86.125000
SHOPERSTOP.NS,CONSUMER SERVICES,150.0,103.0,58.0,NaN,107.0,141.0,NaN,14.0,48.0,NaN,NaN,95.0,89.500000
ALOKINDS.NS,TEXTILES,16.0,28.0,17.0,NaN,35.0,123.0,NaN,1.0,NaN,NaN,NaN,441.0,94.428571
WOCKPHARMA.NS,PHARMA,12.0,27.0,6.0,NaN,41.0,34.0,NaN,213.0,NaN,NaN,NaN,343.0,96.571429
SUNTECK.NS,CONSTRUCTION,25.0,14.0,9.0,NaN,50.0,69.0,NaN,164.0,1.0,NaN,NaN,444.0,97.000000
STAR.NS,PHARMA,79.0,156.0,35.0,NaN,176.0,47.0,NaN,135.0,NaN,NaN,NaN,57.0,97.857143


# Experimentation

In [133]:
imp_financial_df['Sector'].value_counts()

CONSUMER GOODS                       62
INDUSTRIAL MANUFACTURING             41
PHARMA                               37
AUTOMOBILE                           26
IT                                   24
CHEMICALS                            22
CONSTRUCTION                         22
SERVICES                             18
METALS                               18
CONSUMER SERVICES                    17
OIL & GAS                            13
CEMENT & CEMENT PRODUCTS             13
POWER                                12
TEXTILES                             12
FINANCIAL SERVICES                   12
HEALTHCARE SERVICES                  10
FERTILISERS & PESTICIDES             10
TELECOM                               7
MEDIA ENTERTAINMENT & PUBLICATION     6
PAPER AND JUTE                        2
Name: Sector, dtype: int64

In [132]:
# show full dataframe
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# combined_financial_ratio_df.isna().sum() and sort by value
combined_financial_ratio_df.isna().sum().sort_values(ascending=True)

Sector                            0
debt_ratio                      108
return_on_assets                108
net_profit_margin               108
pretax_profit_margin            108
operating_profit_margin         108
effective_tax_rate              109
ROCE                            110
current_ratio                   110
fiftyDayAverage                 111
fiftyTwoWeekLow                 111
fiftyTwoWeekHigh                111
market_cap                      111
twoHundredDayAverage            111
asset_turnover                  111
profitMargins                   111
fixed_asset_turnover            111
interest_coverage_ratio         112
bookValue                       112
enterpriseToRevenue             112
trailingAnnualDividendRate      112
trailing_EPS                    112
gross_profit_margin             112
priceToSalesTrailing12Months    112
DIO                             115
DPO                             115
enterpriseToEbitda              116
cmp                         

In [71]:
# Overall Sector Mean (Higher the better)
imp_financial_score_df.groupby(['Sector']).mean()['final_score'].sort_values()

Sector
SERVICES                    130.500000
POWER                       151.500000
CONSUMER SERVICES           172.500000
HEALTHCARE SERVICES         184.000000
PHARMA                      218.000000
INDUSTRIAL MANUFACTURING    223.000000
OIL & GAS                   225.750000
CONSTRUCTION                229.000000
AUTOMOBILE                  230.500000
CONSUMER GOODS              236.333333
CHEMICALS                   248.000000
TELECOM                     248.000000
METALS                      273.000000
CEMENT & CEMENT PRODUCTS    277.000000
FINANCIAL SERVICES          280.000000
Name: final_score, dtype: float64

In [126]:
backtest_dict = {}
# using yf to download the data calculate if todays close is greater than the close on 31st march 2023 for all the tickers in the index of imp_financial_score_df
for t in imp_financial_score_df.index:
    try:
        temp_df = yf.download(t, start = '2023-03-31', end = '2023-07-13')
        if(temp_df['Close'][-1] > temp_df['Close'][0]):
            backtest_dict[t] = 1
        else:
            backtest_dict[t] = 0
    except:
        print(t, ' Errored out')
        backtest_dict[t] = 0
    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********


1 Failed download:
['INOXLEISUR.NS']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-03-31 -> 2023-07-13)')



INOXLEISUR.NS  Errored out
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


1 Failed download:
['JSLHISAR.NS']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-03-31 -> 2023-07-13)')



JSLHISAR.NS  Errored out
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*******

In [127]:
# count the 1s and 0s in backtest_dict
pd.Series(backtest_dict).value_counts()

1    350
0     34
dtype: int64

# Experimentation

In [5]:
yf.Ticker('INFY.NS').financials

,2023-03-31,2022-03-31,2021-03-31,2020-03-31
Tax Effect Of Unusual Items,9412848.484848,9790882.061447,6764460.51168,0.0
Tax Rate For Calcs,0.276848,0.264618,0.270578,0.244588
Normalized EBITDA,4650000000.0,4492000000.0,4038000000.0,3119000000.0
Total Unusual Items,34000000.0,37000000.0,25000000.0,0.0
Total Unusual Items Excluding Goodwill,34000000.0,37000000.0,25000000.0,0.0
Net Income From Continuing Operation Net Minority Interest,2981000000.0,2963000000.0,2613000000.0,2331000000.0
Reconciled Depreciation,524000000.0,466000000.0,441000000.0,407000000.0
Reconciled Cost Of Revenue,12709000000.0,10996000000.0,8828000000.0,8552000000.0
EBIT,4160000000.0,4063000000.0,3622000000.0,3119000000.0
Net Interest Income,191000000.0,194000000.0,191000000.0,-24000000.0


In [4]:
yf.Ticker('INFY.NS').balance_sheet

,2023-03-31,2022-03-31,2021-03-31,2020-03-31
Treasury Shares Number,12172119.0,13725712.0,15514732.0,18239356.0
Ordinary Shares Number,4136387925.0,4193012929.0,4245146114.0,4240753210.0
Share Issued,4148560044.0,4206738641.0,4260660846.0,4258992566.0
Total Debt,1010000000.0,722000000.0,627000000.0,612000000.0
Tangible Book Value,8077000000.0,8899000000.0,9327000000.0,7696000000.0
...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,2359000000.0,3229000000.0,3700000000.0,3080000000.0
Other Short Term Investments,878000000.0,924000000.0,320000000.0,615000000.0
Cash And Cash Equivalents,1481000000.0,2305000000.0,3380000000.0,2465000000.0
Cash Equivalents,261000000.0,465000000.0,635000000.0,841000000.0


In [3]:
yf.Ticker('INFY.NS').info

{'address1': 'Plot No. 44/97 A',
 'address2': '3rd cross Electronic City Hosur Road',
 'city': 'Bengaluru',
 'zip': '560100',
 'country': 'India',
 'phone': '91 80 2852 0261',
 'fax': '91 80 2852 0362',
 'website': 'https://www.infosys.com',
 'industry': 'Information Technology Services',
 'industryDisp': 'Information Technology Services',
 'sector': 'Technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': "Infosys Limited, together with its subsidiaries, provides consulting, technology, outsourcing, and next-generation digital services in North America, Europe, India, and internationally. It provides application management and application development services, independent validation solutions, product engineering and management, infrastructure management services, traditional enterprise application implementation, support, and integration services. The company's products and platforms include Finacle, a core banking solution; Edge suite of products; Panaya platform, Infosys 